<center><img src='channels4_profile.jpg' height="auto" width="200" /></center>


# SOLVING SUDOKU PUZZLE USING OPENCV

STEPS TO SOLVE SUDOKU PUZZLE USING OPENCV PYTHON:
- LOAD INPUT IMAGE
- LOCALIZE SUDOKU PUZZLE IN INPUT IMAGE
- LOCATE EACH 9 x 9 CELL OF SUDOKU 
- EXTRACT DIGITS, IF PRESENT IN EACH 81 CELLS, OTHERWISE 0
- STORE EXTRACTED DIGITS IN 2D ARRAY OF SIZE 9 x 9
- USE ANY SUDOKU SOLVING ALGORITHM TO SOLVE THE PUZZLE, TAKING 2D ARRAY AS INPUT
- DISPLAY THE RESULT BY ANNOTATING IN THE ORIGINAL IMAGE

### IMPORT ALL REQUIRED LIBRARIES

In [220]:
import numpy as np
import imageio
from imutils import contours
import cv2

### LOAD INPUT IMAGE


In [266]:
src = cv2.imread('image66.png')

In [267]:
cv2.imshow('original image',src)
cv2.waitKey(0)
cv2.destroyAllWindows()

### CONVERT INTO GRAY SCALE IMAGE

In [268]:
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

In [269]:
print(gray)

[[100  89  62 ... 102 102 102]
 [100 255 255 ... 255 255 102]
 [100 255 255 ... 255 255 102]
 ...
 [102 255 255 ... 255 255 102]
 [102 255 255 ... 255 255 102]
 [102 102 102 ... 102 102 102]]


In [270]:
cv2.imshow('gray image',gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

### convert into black and white image

In [271]:
gray = cv2.bitwise_not(gray)
bw = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY, 15, -2)

In [272]:
print(bw)

[[255 255 255 ... 255 255 255]
 [255   0   0 ...   0   0 255]
 [255   0   0 ...   0   0 255]
 ...
 [255   0   0 ...   0   0 255]
 [255   0   0 ...   0   0 255]
 [255 255 255 ... 255 255 255]]


In [273]:
cv2.imshow('black white image',bw)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [172]:
### Extract HORIZONTAL AND VERTICAL LINES

In [274]:
horizontal = np.copy(bw)
vertical = np.copy(bw)

In [275]:
cols = horizontal.shape[1]
horizontal_size = cols // 10

In [276]:
horizontalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (horizontal_size, 1))

In [277]:
horizontal = cv2.erode(horizontal, horizontalStructure)
horizontal = cv2.dilate(horizontal, horizontalStructure)

In [278]:
cv2.imshow('horizontal lines',horizontal)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [279]:
rows = vertical.shape[0]
verticalsize = rows // 10

In [280]:
verticalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (1, verticalsize))

In [281]:
vertical = cv2.erode(vertical, verticalStructure)
vertical = cv2.dilate(vertical, verticalStructure)

In [282]:
cv2.imshow('vertical lines',vertical)
cv2.waitKey(0)
cv2.destroyAllWindows()

### concatenate horizontal and vertical lines to form grid

In [283]:
new_image = cv2.bitwise_or(vertical,horizontal)

In [284]:
cv2.imshow('grids',new_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [285]:
new_im = cv2.bitwise_not(new_image)

In [286]:
edges = cv2.adaptiveThreshold(new_im, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY, 3, -2)

In [287]:
cv2.imshow('concatenated image',edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [288]:
kernel = np.ones((2, 2), np.uint8)
edges = cv2.dilate(edges, kernel)

In [289]:
cv2.imshow('dilated edges',edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [290]:
smooth = cv2.blur(edges, (2, 2))

In [291]:
cv2.imshow('smooth edges',smooth)
cv2.waitKey(0)
cv2.destroyAllWindows()

### find contours of detected edges

In [292]:
cnts, hierarchy = cv2.findContours(image=smooth, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)

In [293]:
cp = src.copy()

In [294]:
cv2.drawContours(image=cp, contours=cnts, contourIdx=-1, color=(0,0,255), thickness=2, lineType=cv2.LINE_AA)


array([[[  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255],
        ...,
        [  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255]],

       [[  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255],
        ...,
        [  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255]],

       [[  0,   0, 255],
        [  0,   0, 255],
        [ 92,  92, 255],
        ...,
        [  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255]],

       ...,

       [[  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255],
        ...,
        [  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255]],

       [[  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255],
        ...,
        [  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255]],

       [[  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255],
        ...,
        [  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255]]

In [295]:
cv2.imshow('detected contours',cp)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [296]:
# sort extracted contours
cnts,_ = contours.sort_contours(cnts,method='left-to-right')
cnts,_ = contours.sort_contours(cnts,method='top-to-bottom')

### plot contours of each cell forming bounding box

In [297]:
cp = src.copy()
count=0
for c in cnts:
#     print(cv2.contourArea(c))
    if(cv2.contourArea(c)>1000 and cv2.contourArea(c)<5000):
        count = count+1
        rect = cv2.boundingRect(c)
        x,y,w,h = rect
        cv2.rectangle(cp,(x,y),(x+w,y+h),(0,255,0),2)
#         cv2.imshow('letters',cp[y:y+h,x:x+w])
        cv2.imshow('black white image',cp)
        cv2.waitKey(500)
cv2.destroyAllWindows()


In [197]:
# cp = src.copy()
# count = 0
# for c in cnts:
#     if(cv2.contourArea(c)>300 and cv2.contourArea(c)<1000):
#         count = count+1
#         rect = cv2.boundingRect(c)
#         x,y,w,h = rect
#         cv2.rectangle(cp,(x,y),(x+w,y+h),(0,255,0),2)
# #         cv2.imshow('letters',cp[y:y+h,x:x+w])
#         cv2.imshow('black white image',cp)
#         cv2.waitKey(500)
# cv2.destroyAllWindows()

### save each cell within bounding box as image. This image will be used as input for trained CNN model for prediction

In [198]:
# import imageio

# cp = src.copy()
# count = 0
# for c in cnts:
#     if(cv2.contourArea(c)>300 and cv2.contourArea(c)<1000):
#         count = count+1
#         rect = cv2.boundingRect(c)
#         x,y,w,h = rect
#         array = np.array(cp[y:y+h,x:x+w])
#         imageio.imwrite('numbers/image_{}.jpeg'.format(count),array)
#         print(array)

### use pytesseract for OCR (optical character recognition)
### I will also be demonstrating, the use of CNN-model for OCR

In [298]:
import pytesseract
import os

def convert_gray(img):
    im = cv2.imread(img)
    return cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
# custom_config = r'--oem 3 --psm 6 outputbase digits'
def get_num(img):
    return pytesseract.image_to_string(img, config=r'--oem 3 --psm 6 outputbase digits')

In [299]:
import imageio

cp = src.copy()
count = 0
num= []
for c in cnts:
    if(cv2.contourArea(c)>1000 and cv2.contourArea(c)<5000):
        count = count+1
        rect = cv2.boundingRect(c)
        x,y,w,h = rect
        array = np.array(cp[y:y+h,x:x+w])
        num.append(get_num(array))
#         imageio.imwrite('numbers/image_{}.jpeg'.format(count),array)
#         print(array)

In [201]:
# import pytesseract
# import os

In [202]:
# def convert_gray(img):
#     im = cv2.imread(img)
#     return cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
# # custom_config = r'--oem 3 --psm 6 outputbase digits'
# def get_num(img):
#     return pytesseract.image_to_string(img, config=r'--oem 3 --psm 6 outputbase digits')

In [203]:
# num = []
# i=1
# for file in os.listdir('numbers/'):
# #     print('numbers/image_{}'.format(i)+'.jpeg')
#     num.append(get_num(convert_gray('numbers/image_{}'.format(i)+'.jpeg')))
#     i = i+1

In [300]:
len(num)

81

In [301]:
def divide_chunks(l, n):
    for i in range(0, len(l), n): 
        yield l[i:i + n]
  
n = 9
matrix = list(divide_chunks(num, n))
print (matrix)

[['7\n\x0c', '8\n\x0c', '\x0c', '4\n\x0c', '\x0c', '\x0c', '1\n\x0c', '2\n\x0c', '\x0c'], ['6\n\x0c', '\x0c', '\x0c', '\x0c', '7\n\x0c', '5\n\x0c', '\x0c', '\x0c', '9\n\x0c'], ['\x0c', '\x0c', '\x0c', '6\n\x0c', '\x0c', '1\n\x0c', '\x0c', '7\n\x0c', '8\n\x0c'], ['\x0c', '\x0c', '7\n\x0c', '\x0c', '4\n\x0c', '\x0c', '2\n\x0c', '6\n\x0c', '\x0c'], ['\x0c', '\x0c', '1\n\x0c', '\x0c', '5\n\x0c', '\x0c', '9\n\x0c', '3\n\x0c', '\x0c'], ['9\n\x0c', '\x0c', '4\n\x0c', '\x0c', '6\n\x0c', '\x0c', '\x0c', '\x0c', '5\n\x0c'], ['\x0c', '7\n\x0c', '\x0c', '3\n\x0c', '\x0c', '\x0c', '\x0c', '1\n\x0c', '2\n\x0c'], ['1\n\x0c', '2\n\x0c', '\x0c', '\x0c', '\x0c', '7\n\x0c', '4\n\x0c', '\x0c', '\x0c'], ['\x0c', '4\n\x0c', '9\n\x0c', '2\n\x0c', '\x0c', '6\n\x0c', '\x0c', '\x0c', '7\n\x0c']]


In [302]:
matrix[0][0]

'7\n\x0c'

In [303]:
import copy
mat = copy.deepcopy(matrix)
for n in mat:
    for j,i in enumerate(n):
        if i=='\x0c':
            n[j]=i.replace('\x0c','0')
        else:
            n[j]=i.replace(i,i[0])
            

In [304]:
mat

[['7', '8', '0', '4', '0', '0', '1', '2', '0'],
 ['6', '0', '0', '0', '7', '5', '0', '0', '9'],
 ['0', '0', '0', '6', '0', '1', '0', '7', '8'],
 ['0', '0', '7', '0', '4', '0', '2', '6', '0'],
 ['0', '0', '1', '0', '5', '0', '9', '3', '0'],
 ['9', '0', '4', '0', '6', '0', '0', '0', '5'],
 ['0', '7', '0', '3', '0', '0', '0', '1', '2'],
 ['1', '2', '0', '0', '0', '7', '4', '0', '0'],
 ['0', '4', '9', '2', '0', '6', '0', '0', '7']]

In [305]:
for row in mat:
    for i,j in enumerate(row):
        row[i] = int(j)

In [306]:
mat

[[7, 8, 0, 4, 0, 0, 1, 2, 0],
 [6, 0, 0, 0, 7, 5, 0, 0, 9],
 [0, 0, 0, 6, 0, 1, 0, 7, 8],
 [0, 0, 7, 0, 4, 0, 2, 6, 0],
 [0, 0, 1, 0, 5, 0, 9, 3, 0],
 [9, 0, 4, 0, 6, 0, 0, 0, 5],
 [0, 7, 0, 3, 0, 0, 0, 1, 2],
 [1, 2, 0, 0, 0, 7, 4, 0, 0],
 [0, 4, 9, 2, 0, 6, 0, 0, 7]]

In [307]:
grid= copy.deepcopy(mat)

## puzzle solving 

In [308]:
M = 9
def puzzle(a):
    for i in range(M):
        for j in range(M):
            print(a[i][j],end = " ")
        print()
def solve(grid, row, col, num):
    for x in range(9):
        if grid[row][x] == num:
            return False
             
    for x in range(9):
        if grid[x][col] == num:
            return False
 
 
    startRow = row - row % 3
    startCol = col - col % 3
    for i in range(3):
        for j in range(3):
            if grid[i + startRow][j + startCol] == num:
                return False
    return True
 
def Suduko(grid, row, col):
 
    if (row == M - 1 and col == M):
        return True
    if col == M:
        row += 1
        col = 0
    if grid[row][col] > 0:
        return Suduko(grid, row, col + 1)
    for num in range(1, M + 1, 1): 
     
        if solve(grid, row, col, num):
         
            grid[row][col] = num
            if Suduko(grid, row, col + 1):
                return True
        grid[row][col] = 0
    return False


if (Suduko(grid, 0, 0)):
    print(grid)
#     puzzle(grid)
else:
    print("Solution does not exist:(")

[[7, 8, 5, 4, 3, 9, 1, 2, 6], [6, 1, 2, 8, 7, 5, 3, 4, 9], [4, 9, 3, 6, 2, 1, 5, 7, 8], [8, 5, 7, 9, 4, 3, 2, 6, 1], [2, 6, 1, 7, 5, 8, 9, 3, 4], [9, 3, 4, 1, 6, 2, 7, 8, 5], [5, 7, 8, 3, 9, 4, 6, 1, 2], [1, 2, 6, 5, 8, 7, 4, 9, 3], [3, 4, 9, 2, 1, 6, 8, 5, 7]]


In [309]:
bound = []
for c in cnts:
    if(cv2.contourArea(c)>1000 and cv2.contourArea(c)<5000):
        count = count+1
        rect = cv2.boundingRect(c)
        bound.append(rect)

In [310]:
len(bound)

81

In [311]:
grid

[[7, 8, 5, 4, 3, 9, 1, 2, 6],
 [6, 1, 2, 8, 7, 5, 3, 4, 9],
 [4, 9, 3, 6, 2, 1, 5, 7, 8],
 [8, 5, 7, 9, 4, 3, 2, 6, 1],
 [2, 6, 1, 7, 5, 8, 9, 3, 4],
 [9, 3, 4, 1, 6, 2, 7, 8, 5],
 [5, 7, 8, 3, 9, 4, 6, 1, 2],
 [1, 2, 6, 5, 8, 7, 4, 9, 3],
 [3, 4, 9, 2, 1, 6, 8, 5, 7]]

In [312]:
mat

[[7, 8, 0, 4, 0, 0, 1, 2, 0],
 [6, 0, 0, 0, 7, 5, 0, 0, 9],
 [0, 0, 0, 6, 0, 1, 0, 7, 8],
 [0, 0, 7, 0, 4, 0, 2, 6, 0],
 [0, 0, 1, 0, 5, 0, 9, 3, 0],
 [9, 0, 4, 0, 6, 0, 0, 0, 5],
 [0, 7, 0, 3, 0, 0, 0, 1, 2],
 [1, 2, 0, 0, 0, 7, 4, 0, 0],
 [0, 4, 9, 2, 0, 6, 0, 0, 7]]

In [313]:
n = 9
bound = list(divide_chunks(bound, n))
print (bound)

[[(4, 34, 57, 57), (64, 34, 57, 57), (124, 34, 56, 57), (186, 34, 55, 57), (244, 34, 57, 57), (304, 34, 56, 57), (366, 34, 55, 57), (424, 34, 57, 57), (484, 34, 56, 57)], [(4, 94, 57, 57), (64, 94, 57, 57), (124, 94, 56, 57), (186, 94, 55, 57), (244, 94, 57, 57), (304, 94, 56, 57), (366, 94, 55, 57), (424, 94, 57, 57), (484, 94, 56, 57)], [(4, 154, 57, 56), (64, 154, 57, 56), (124, 154, 56, 56), (186, 154, 55, 56), (244, 154, 57, 56), (304, 154, 56, 56), (366, 154, 55, 56), (424, 154, 57, 56), (484, 154, 56, 56)], [(4, 216, 57, 55), (64, 216, 57, 55), (124, 216, 56, 55), (186, 216, 55, 55), (244, 216, 57, 55), (304, 216, 56, 55), (366, 216, 55, 55), (424, 216, 57, 55), (484, 216, 56, 55)], [(4, 274, 57, 57), (64, 274, 57, 57), (124, 274, 56, 57), (186, 274, 55, 57), (244, 274, 57, 57), (304, 274, 56, 57), (366, 274, 55, 57), (424, 274, 57, 57), (484, 274, 56, 57)], [(4, 334, 57, 56), (64, 334, 57, 56), (124, 334, 56, 56), (186, 334, 55, 56), (244, 334, 57, 56), (304, 334, 56, 56), (366

In [314]:
cp = src.copy()
for i,m in enumerate(mat):
    for j,n in enumerate(m):
        if(mat[i][j]!=grid[i][j]):
            x,y,w,h = bound[i][j]
#             cv2.rectangle(cp,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.putText(cp,str(grid[i][j]),(x+w-20,y+h-20),0,0.9,(0,0,255))          

In [316]:
cv2.imshow('im',cp)
cv2.waitKey(0)
cv2.destroyAllWindows()